In [1]:
import numpy as np
import pandas as pd
from PIL import Image
import json 
import cv2

In [4]:
path='./dataset/gen_img_data.json'

In [5]:
with open(path, 'r') as f:
  datastore = json.load(f)

In [6]:
print(len(datastore))

23702


In [7]:
def get_values_class(json_data):
    labels=[]
    pixel_values=[]
    for i in range(len(json_data)):
    
        for name, value in json_data[i].items():    # for name, age in dictionary.iteritems():  (for Python 2.x)
            labels.append(name)
      # print(name)

            for name, value in value.items():
                pixel_values.append(value)
        # print(len(value))
        # print(value)

    return (pixel_values, labels)

In [8]:
X,y=get_values_class(datastore)

In [9]:
df=pd.DataFrame(list(zip(X,y)),columns =['X', 'labels'])

In [11]:
X_data = []

for i in range(len(df['X'])):
    image=df['X'][i]
    image=np.asarray(image)

    if image.shape[0]==1024:
    # image.shape=(32,32)
        df['X'][i] = image.reshape((32,32)).astype('float32')
    # image = cv2.resize(image, dsize=(28,28), interpolation=cv2.INTER_CUBIC)

    elif image.shape[0]==784:

        image = image.reshape((28,28)).astype('float32')

        df['X'][i] =  cv2.resize(image, dsize=(32,32), interpolation=cv2.INTER_CUBIC)

    # image = image.reshape((28,28)).astype('float32')

    # image.shape=(28,28)

  # X_data[i]=image
  

In [12]:
for i in range(len(df['X'])):
    image=df['X'][i]
    name="./dataset/"+str(i)+'.png'
    cv2.imwrite(name,image)

In [13]:
import cv2
import glob
images = np.asarray([cv2.imread(file) for file in glob.glob('./dataset/*.png')])

In [14]:
image_np_array=np.zeros((len(images),32,32,3),dtype=float)

In [15]:
image_np_array.shape

(23702, 32, 32, 3)

In [16]:
images.shape

(23702, 32, 32, 3)

In [18]:
for i in range(len(images)):

    for x in range(images[i].shape[0]):
        for y in range(images[i].shape[1]):
            for z in range(images[i].shape[2]):
                image_np_array[i][x][y][z]=images[i][x][y][z]

In [19]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(df['labels'])

LabelEncoder()

In [20]:
y_transform=le.transform(df['labels'])

In [21]:
from tensorflow.keras import utils as np_utils
Y_train = np_utils.to_categorical(y_transform, num_classes = 6)

In [22]:
from sklearn.model_selection import train_test_split

In [23]:
trainX, testX, trainY, testY = train_test_split(image_np_array,Y_train, test_size=0.2, random_state=42)

In [24]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D,Flatten,Dropout

Using TensorFlow backend.


In [25]:
model = Sequential()
#convolutional layer with rectified linear unit activation
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(32,32,3)))
#32 convolution filters used each of size 3x3
#again
model.add(Conv2D(64, (3, 3), activation='relu'))
#64 convolution filters used each of size 3x3
#choose the best features via pooling
model.add(MaxPooling2D(pool_size=(2, 2)))
#randomly turn neurons on and off to improve convergence
model.add(Dropout(0.25))
#flatten since too many dimensions, we only want a classification output
model.add(Flatten())
#fully connected to get all relevant data
model.add(Dense(128, activation='relu'))
#one more dropout for convergence' sake :) 
model.add(Dropout(0.5))
#output a softmax to squash the matrix into output probabilities
model.add(Dense(6, activation='softmax'))

In [26]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 12544)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1605760   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)              

In [29]:
import keras
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

In [30]:
batch_size = 128
num_epoch = 10
#model training
model_log = model.fit(trainX, trainY,
          batch_size=batch_size,
          epochs=num_epoch,
          verbose=1,
          validation_data=( testX,  testY))

Train on 18961 samples, validate on 4741 samples
Epoch 1/10
18961/18961 [==============================] - 22s 1ms/step - loss: 11.6928 - accuracy: 0.2305 - val_loss: 1.6123 - val_accuracy: 0.2449
Epoch 2/10
18961/18961 [==============================] - 19s 1ms/step - loss: 1.6288 - accuracy: 0.2543 - val_loss: 1.5961 - val_accuracy: 0.2512
Epoch 3/10
18961/18961 [==============================] - 20s 1ms/step - loss: 1.5974 - accuracy: 0.2668 - val_loss: 1.5900 - val_accuracy: 0.2428
Epoch 4/10
18961/18961 [==============================] - 20s 1ms/step - loss: 1.5860 - accuracy: 0.2681 - val_loss: 1.5877 - val_accuracy: 0.2440
Epoch 5/10
18961/18961 [==============================] - 20s 1ms/step - loss: 1.5769 - accuracy: 0.2809 - val_loss: 1.5911 - val_accuracy: 0.2464
Epoch 6/10
18961/18961 [==============================] - 20s 1ms/step - loss: 1.5661 - accuracy: 0.2884 - val_loss: 1.5842 - val_accuracy: 0.2478
Epoch 7/10
18961/18961 [==============================] - 19s 997us/